# YugiohNet

### Yugioh Archetype Classification

### Created by Conrad Smith

In [ ]:
import os
import shutil
import json
import requests
from PIL import Image, ImageFile
from io import BytesIO

import matplotlib.pyplot as plt

import numpy as np
from numpy.random import seed

import optuna

import pandas as pd

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LambdaCallback, CSVLogger
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, \
                         BatchNormalization, Conv2D, MaxPooling2D, ReLU, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Progbar

# ensures GPU doesn't crash mid training
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# necessary for creating dataset
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Global Environment

#### Hyperparameter / Constants Storage

In [20]:
class GlobalEnvironment:
    def __init__(self, 
                 data_url: str, 
                 data_path: str, 
                 data_file_name: str,
                 experiment_name: str,
                 load_cp: bool = False,
                 load_cp_and_train: bool = True,
                 load_model: bool = False,
                 seed: int = 1,
                 image_size = (224, 224)):
        self.data_url = data_url
        self.data_path = data_path
        self.data_file_name = data_file_name
        self.seed = seed
        self.image_size = image_size
        self.experiment_name = experiment_name
        self.load_cp = load_cp
        self.load_cp_and_train = load_cp_and_train
        self.load_model = load_model
        
        self.data_file_path = os.path.join(data_path, data_file_name)
        self.class_number = len(next(os.walk(self.data_path))[1])
        
        # parameter space for optuna
        self.p_optuna = {
            'activation': ('leakyrelu', 'relu'),
            'optimizer': ('Adam', 'Nadam', 'RMSprop', 'SGD'),
            'batch_size': (16, 32, 64),
            'leaky_alpha': (0.001, 0.3),
            'lr': (1e-5, 1e-1),
            'epochs': 8
        }
        
        # parameter space not involved in hyperparameter tuning
        self.p_const = {
            'activation': 'relu',
            'optimizer': 'RMSProp',
            'batch_size': 32,
            'epochs': 200,
            'leaky_alpha': 0.1,
            'lr': 0.001
        }

        
Global = GlobalEnvironment(experiment_name = 'vanilla',
                           data_url = 'https://db.ygoprodeck.com/api/v7/cardinfo.php',
                           data_path = 'data',
                           data_file_name = 'archetypes.csv')
np.random.seed(Global.seed)
tf.random.set_seed(Global.seed)

In [ ]:
r = requests.get(Global.data_url)
data = r.json()
pdata = json.dumps(data, indent=4)
# Print the type of data variable
print("Type:", type(data))
print(data['data'][0]['archetype'])

# Print the data of dictionary
print("\nArchetype Card:", json.dumps(data['data'][0], indent=4))
print("\nNon-Archetype Card:", json.dumps(data['data'][8], indent=4))

## Dataset Creation

In [ ]:
# r = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
# data = r.json()
def create_dataset():
    cards = data['data']
    n = len(cards)
    prog = Progbar(n)
    # iterate through every card; index needed for progress bar
    for i in range(n):
        card = cards[i]
        
        # some cards do not have an archetype field; label it as 'No_Archetype' (this will cause an error otherwise)
        # also, some archetypes have slashes which is problematic. replace those with underscores
        archetype_path = os.path.join(Global.data_path, (card['archetype'] if 'archetype' in card else 'No_Archetype').replace("/", "_"))
        # make directory if archetype doesn't exist
        if not os.path.isdir(archetype_path):
            os.mkdir(archetype_path)
            
        for image in card['card_images']:
            file_path = os.path.join(archetype_path, '{}.png'.format(image['id']))
            # if we've fetched the image for this card ID previously, skip it
            if not os.path.exists(file_path):
                url = image['image_url']
                # fetch raw picture content from URL
                # https://stackoverflow.com/a/18043472
                response = requests.get(url, stream=True)
                with open(file_path, 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
                    Image.open(file_path).resize(Global.image_size).save(file_path)
        
        # update progress bar so we know how long this is going to take
        prog.update(i)
    # finish progress bar
    prog.update(n, finalize=True)
    
# create the CSV
def generate_csv():
    cards = pd.DataFrame(data['data'])
    print(cards.loc[0, ['archetype']])
    cards['archetype'] = cards['archetype'].fillna('No_Archetype')
    # cards = cards.loc[:, ['card_images'[:, 'id'], 'archetype']]
    # cards = cards['id', 'archetype']
    entries = []
    for i in range(len(cards)):
        card = cards.loc[i]
        for j in range(len(card.loc['card_images'])):
            image = card.loc['card_images'][j]
        entries.append([os.path.join(Global.data_path, card['archetype'], '{}.png'.format(image['id'])), image['id'], card['archetype']])

    entries = pd.DataFrame(entries)
    for i in range(10):
        print(entries.iloc[i])
    entries.columns = ['file_path', 'id', 'archetype']
    entries.to_csv(Global.data_file_path, index=False)

def generate_freqs():
    freqs = pd.read_csv(Global.data_file_path, delimiter=',').value_counts(subset=['archetype'])
    # freqs = pd.DataFrame(freqs)
    print(freqs[:10])
    freqs.plot(ylim=(0, freqs[1]))
    
def resize_images():
    cards = data['data']
    n = len(cards)
    prog = Progbar(n)
    # iterate through every card; index needed for progress bar
    for i in range(n):
        card = cards[i]
        
        # some cards do not have an archetype field; label it as 'No_Archetype' (this will cause an error otherwise)
        # also, some archetypes have slashes which is problematic. replace those with underscores
        archetype_path = os.path.join(Global.data_path, (card['archetype'] if 'archetype' in card else 'No_Archetype').replace("/", "_"))
        # make directory if archetype doesn't exist
        if not os.path.isdir(archetype_path):
            os.mkdir(archetype_path)
            
        for image in card['card_images']:
            file_path = os.path.join(archetype_path, '{}.png'.format(image['id']))
            # if we've fetched the image for this card ID previously, skip it
            if os.path.exists(file_path):
                Image.open(file_path).resize(Global.image_size).save(file_path)
        
        # update progress bar so we know how long this is going to take
        prog.update(i)
    # finish progress bar
    prog.update(n, finalize=True)
        
# create_dataset()
# generate_csv()
generate_freqs()
# resize_images()

## TF Data Generator

In [ ]:
def generate_data(data_generator, batch_size, subset):
    generator = data_generator.flow_from_directory(
        directory = Global.data_path,
        target_size = Global.image_size,
        batch_size = batch_size,
        shuffle = True,
        seed = Global.seed,
        subset = subset
    )
    
    return generator

## Hyperparameter Tuning / Fine Tuning / Model Training

In [15]:
def objective(trial):
   # ---------- Model Hyperparameters ----------
    batch_size = Global.p_const['batch_size']
    # dropout = Global.p_const['dropout']
    lr = Global.p_const['lr']
    optim = tf.keras.optimizers.get({"class_name": Global.p_const['optimizer'], 
                                     "config": {"learning_rate": lr}})
    activation_layer = LeakyReLU(alpha = Global.p_const['leaky_alpha']) if Global.p_const['activation'] == 'leakyrelu' else ReLU()
    
    # if conducting an optuna study
    if trial != None:
        batch_size = trial.suggest_categorical('batch_size', Global.p_optuna['batch_size'])
        lr = trial.suggest_float('lr', Global.p_optuna['lr'][0], Global.p_optuna['lr'][1])
        optim = tf.keras.optimizers.get({"class_name": trial.suggest_categorical('optimizer', Global.p_optuna['optimizer']), 
                                         "config": {"learning_rate": lr}})

        activation_layer = trial.suggest_categorical('activation', Global.p_optuna['activation'])
        if activation_layer == 'leakyrelu':
            activation_layer = LeakyReLU(alpha = trial.suggest_float('leaky_alpha', Global.p_optuna['leaky_alpha'][0], Global.p_optuna['leaky_alpha'][1]))
        else:
            activation_layer = ReLU()
    
    # ---------- Model Architecture ----------
    model = Sequential()
    if Global.load_model:
        # skip training loop and just return model
        model = tf.keras.models.load_model('./models/' + Global.experiment_name, compile = False)
        return model
    else:
        resnet = ResNet50(
            include_top = False, 
            weights = None, 
            input_shape = (*Global.image_size, 3), 
            pooling = 'max'
        )
        for l in resnet.layers:
            l.trainable = True
        model.add(resnet)
        # model.add(Flatten(input_shape = resnet.output_shape[1:]))
        model.add(Dense(
            Global.class_number, 
            kernel_initializer=GlorotUniform(seed = Global.seed),
            activation='softmax'))

        # compile 

    model.compile(
        optimizer = optim, 
        loss = tf.keras.losses.CategoricalCrossentropy(), 
        metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Recall()]
    )

    # ---------- Callbacks ----------
    # set up checkpoints
    # https://www.tensorflow.org/tutorials/keras/save_and_load
    checkpoint_path = "experiments/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        save_weights_only = True,     
        save_best_only = True,
        verbose = 1
    )
    early_stopper = EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        patience = 20, 
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    lr_reducer = ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 10,
        verbose = 1,
        mode = 'min',
        min_delta = 1e-3
    )
    csv_logger = CSVLogger(
        filename = Global.experiment_name + '.csv',
    )

    # only want pruning callback if using optuna, as others aren't necessary for low epoch runs
    callbacks = (cp_callback, early_stopper, lr_reducer, csv_logger) if trial is None else \
                (optuna.integration.TFKerasPruningCallback(trial, 'val_loss'))

    # ---------- Loading, Training, Evaluation ----------
    # don't need to train if just loading weights, but we did need to recompile the model
    # in order to load weights
    if Global.load_cp:
        model.load_weights(checkpoint_path)
        if Global.load_cp_and_train == False:
            return model
    
    # train    
    data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_generator = generate_data(data_generator, batch_size, subset='training')
    valid_generator = generate_data(data_generator, batch_size, subset='validation')

    if trial != None:
        print('Trial Number: ', trial.number)
        print('Trial Parameters: ', trial.params)
    out = model.fit(
        x = train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size, # for some reason tf won't do this automatically here
        validation_data = valid_generator,
        validation_steps = valid_generator.n // valid_generator.batch_size,
        epochs = Global.p_const['epochs'] if trial is None else Global.p_optuna['epochs'],
        callbacks = callbacks,
        workers = 6,
        verbose = 1
    )

    # evaluate
    results = model.evaluate(
        valid_generator, 
        verbose = 1, 
        use_multiprocessing = True,
        steps = valid_generator.n // valid_generator.batch_size,
        workers = 6
    )
    if trial is None:
        model.save('./models/' + Global.experiment_name)
    
    print (results)
    return results[0] if trial is not None else model # return validation loss

## Optuna Study Functions

In [16]:
# generate some figures related to a completed optuna study
def report(study):
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()
    fig = optuna.visualization.plot_intermediate_values(study)
    # fig.update_layout(yaxis_range=[0, 8100]) # remove outliers
    fig.show()
    fig = optuna.visualization.plot_contour(study, params = ['dropout', 'lr'])
    fig.show()
    fig = optuna.visualization.plot_param_importances(study)
    fig.show() 
    
    # https://github.com/optuna/optuna-examples/blob/main/tensorflow/tensorflow_eager_simple.py
    trials = study.best_trials
    for t in trials:
        print("Trial: ", t.number, ", Validation Loss: ", t.value)
        print("Params: ")
        for key, value in t.params.items():
            print("    {}: {}".format(key, value))

# conduct an optuna study
def conduct_study(study_name):
    with tf.device('/device:GPU:0'): # utilizes the GPU of the remote server
        store = 'sqlite:///' + study_name + '.db'
        study = optuna.create_study(
            study_name = study_name,
            direction = 'minimize',
            pruner = optuna.pruners.MedianPruner(),
            storage = store
        )
        study.optimize(objective, n_trials = 100)
        print('Done!')
        
# evaluate a completed optuna study
def eval_study(study_name):
    with tf.device('/device:GPU:0'):
        store = 'sqlite:///' + study_name + '.db'
        study = optuna.load_study(study_name = study_name,
                                     storage = 'sqlite:///RISENet-SC-Study.db',
                                     pruner = optuna.pruners.MedianPruner())
        report(study)
        print('Done!')

In [ ]:
# if tf.test.gpu_device_name():
#     print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# else:
#     print("Please install GPU version of TF")

conduct_study(Global.experiment_name)
#     model = objective(None)

## Load a Saved Model

In [ ]:
# load_model doesn't work with compile=True so I have to compile manually
model = tf.keras.models.load_model('./models/' + Global.experiment_name, compile = False)
model.compile(optimizer = tf.keras.optimizers.get({"class_name": Global.p_const['optimizer'], 
                                     "config": {"learning_rate": Global.p_const['lr']}}), 
              loss = M.circular_mse, 
              metrics = [M.circular_mae, M.mean_angular_error]
)